<a href="https://colab.research.google.com/github/anko191/Python_Kaggle/blob/master/Tensorflow/Try_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
    #optimizer = tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period = 10, 
    #                                    name = 'Lookahead_10')
    # optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    #optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
    # optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    # optimizer = keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
    # optimizer = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=1e-4)
    optimizer = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-4, amsgrad=False)
    # optimizer = tfa.optimizers.Lookahead(keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-4, amsgrad=False),                                         sync_period = 10, name = 'Lookahead')

## 実行
* Importance_col only
    * [ - 全部標準化しちゃった -]
        - glorot_uniform
            - 4096 loss 1.000 so slow 80s
            * 2048 loss ?
            * 2-layer ReLU 1024 loss first 0.3874 fast 7s -> 0.0484 
            * 3-layer ReLU 1024 loss first 0.2645 fast -> 0.0499
            * 2-layer ReLU 512 loss first 0.4885 -> 0.0449
                * Dropout(0.2)
            * 3-layer ReLU 512 loss first 0.3811 -> ?
    * [ - startswith('cp-')のだけ -]
        - glorot_uniform
            * 3-layer ReLU 512 loss first 0.5000 -> 0.0473
                * Dropout(0.2)
            * 2-layer ReLU 512 loss first 0.5762 -> 0.0484
                * Dropout(0.2)
            * 2-layer ReLU 2048 loss first 0.5  -> 0.0469
                * Dropout(0.5)
        - not designate initializer
            * 2-layer ReLU 2048 loss first 0.58 -> 0.0472
     * 10/8 all normalization
         * optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-4, amsgrad=False)
            * ReLU 1024 layer + sigmoid 206 layer
                * 0.0695 -> 0.0165 ...
         * tfa.forward below model ↑
            * same
                * 0.30 -> 0.0165
* all_columns
    * [ - startswith('cp-')のだけ -]
        - not designate initializer
            * 2-layer ReLU 2048 loss first 0.5988 -> 0.0472
            * 2-layer ReLU 1024 loss first 0.4805 -> 0.0494
        * Adam
            * 2-layer 1024 relu 0.4 -> 0.04
        * sgd keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=1e-4, nesterov=True)
            * 2l 1024 relu 0.13 -> 0.13 ... orz
        * keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
            * 0.0541 !? -> 0.0525
        * keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
            * 0.4108 -> 0.0471
        * keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
            * 0.4173 -> 0.0474
        * keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=1e-4)
            * 0.2879 -> 0.0462
    * 全部をNormalization
        * Optimizer : Adadelta lr = 1.0, rho = 0.95, decay 1e-4 epsilon = None
            * 206 のレイヤーのみ。てかsigmoidどっかで入れなあかんのでは？
                * 0.28 -> 0.0443
            * sigmoidにするか？
                * 0.43 -> 0.0325 !? 
            * ReLU 1024 layer + sigmoid 206 layer
                * 0.43 -> 0.0181 !??!?!?! yabee
        * Optimzier : Adam 
        * optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-4, amsgrad=False)
            * ReLU 1024 layer + sigmoid 206 layer
                * 0.44 -> 0.0127 !? leakage..?
                * 0.44 -> 0.0164 -> 0.0160
         * tfa.forward below model ↑
            * same below
                * 0.3124 -> 0.0162]
         * not using...
            * sigmoid 1024 -> ReLU 1024 -> sigmoid
                * 0.060 -> 0.0162
            * ReLU 1024 -> ReLU 1024 -> sigmoid
                * 0.06 -> 0.0163
            * sigmoid 206 only...
                * 0.6 -> 0.05 No
            * ReLU 206 only
                * 0.6 -> 0.0476
       * optimizer = keras.optimizers.Adam(★lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=1e-4, amsgrad=False)
         * not using...
            * sigmoid 1024 -> ReLU 1024 -> sigmoid
                * 